In [48]:
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import K2Score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

## Создаём синтетический датасет

In [49]:
import numpy as np
import pandas as pd

# Задаем события
events = ['B', 'E', 'A', 'J', 'M']

# Задаем вероятности для событий B и E
probabilities_B = [0.001, 0.999]
probabilities_E = [0.002, 0.998]

# Генерируем события B и E
B_outcomes = np.random.choice(['+', '-'], size=20000, p=probabilities_B)
E_outcomes = np.random.choice(['+', '-'], size=20000, p=probabilities_E)

# Задаем таблицу условных вероятностей для события A при условии B и E
conditional_probabilities_A = {
    ('+', '+'): {'+': 0.95, '-': 0.05},
    ('+', '-'): {'+': 0.94, '-': 0.06},
    ('-', '+'): {'+': 0.29, '-': 0.71},
    ('-', '-'): {'+': 0.001, '-': 0.999}
}

# Задаем таблицы условных вероятностей для событий J и M при условии A
conditional_probabilities_J = {'+': {'+': 0.9, '-': 0.1}, '-': {'+': 0.05, '-': 0.95}}
conditional_probabilities_M = {'+': {'+': 0.7, '-': 0.3}, '-': {'+': 0.01, '-': 0.99}}

# Генерируем события A, J и M
A_outcomes = []
J_outcomes = []
M_outcomes = []

for i in range(20000):
    B_event = B_outcomes[i]
    E_event = E_outcomes[i]
    
    # Генерация A
    A_event = np.random.choice(['+', '-'], p=[conditional_probabilities_A[(B_event, E_event)]['+'], conditional_probabilities_A[(B_event, E_event)]['-']])
    
    # Генерация J
    J_event = np.random.choice(['+', '-'], p=[conditional_probabilities_J[A_event]['+'], conditional_probabilities_J[A_event]['-']])
    
    # Генерация M
    M_event = np.random.choice(['+', '-'], p=[conditional_probabilities_M[A_event]['+'], conditional_probabilities_M[A_event]['-']])
    
    A_outcomes.append(A_event)
    J_outcomes.append(J_event)
    M_outcomes.append(M_event)

# Создаем DataFrame
df = pd.DataFrame({'B': B_outcomes, 'E': E_outcomes, 'A': A_outcomes, 'J': J_outcomes, 'M': M_outcomes})

# Выводим синтетический датасет
df

,B,E,A,J,M
0,-,-,-,-,-
1,-,-,-,-,-
2,-,-,-,-,-
3,-,-,-,-,+
4,-,-,-,-,-
...,...,...,...,...,...
19995,-,-,-,-,-
19996,-,-,-,+,-
19997,-,-,-,-,-
19998,-,-,-,-,-


## Определяем структуру сети

In [50]:
# Задаем данные
data = df[['B', 'E', 'A', 'J', 'M']]

# Инициализируем алгоритм Hill Climbing
hc = HillClimbSearch(data)

# Запускаем алгоритм для поиска оптимальной структуры
best_model = hc.estimate(scoring_method=K2Score(data))

# Выводим найденную структуру
print("Найденная структура:")
print(best_model.edges())

  0%|          | 0/1000000 [00:00<?, ?it/s]

Найденная структура:
[('B', 'A'), ('E', 'A'), ('A', 'M'), ('A', 'J')]


## Создаём, обучаем и тестируем Байесовскую сеть

In [51]:
# Создаем байесовскую сеть с найденной оптимальной структурой
model = BayesianNetwork(best_model.edges())

# Разделяем данные на обучающую и тестовую выборки
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Обучаем параметры байесовской сети на обучающей выборке
model.fit(train_data, estimator=BayesianEstimator)

# Предсказываем значения для тестовой выборки
predicted_data = model.predict(test_data.drop(columns=['J', 'M']))

# Оцениваем точность прогнозирования
accuracy_J = accuracy_score(test_data['J'], predicted_data['J'])
accuracy_M = accuracy_score(test_data['M'], predicted_data['M'])

print(f"Точность прогнозирования для события J: {accuracy_J * 100:.2f}%")
print(f"Точность прогнозирования для события M: {accuracy_M * 100:.2f}%")

  0%|          | 0/5 [00:00<?, ?it/s]

Точность прогнозирования для события J: 94.30%
Точность прогнозирования для события M: 99.00%
